In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from pprint import pprint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix

Mounted at /content/drive


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/DS5500 PROJECT/processed_train.csv', low_memory=False)
test_df = pd.read_csv('/content/drive/MyDrive/DS5500 PROJECT/processed_test.csv', low_memory=False)
pprint(f'Train dataset shape: {train_df.shape}')
pprint(f'Test dataset shape: {test_df.shape}')

'Train dataset shape: (1025602, 221)'
'Test dataset shape: (134529, 221)'


In [3]:
columns_remove = ["Have you noticed a diffuse (widespread) redness in one or both eyes?",
         "Have you lost consciousness associated with violent and sustained muscle contractions or had an absence episode?",
         "Have you had any vaginal discharge?"]

In [4]:
train_df = train_df.drop(columns=columns_remove)
test_df = test_df.drop(columns=columns_remove)

In [5]:
train_df = train_df.dropna()
test_df = test_df.dropna()
pprint(f'Train dataset shape: {train_df.shape}')
pprint(f'Test dataset shape: {test_df.shape}')

'Train dataset shape: (982224, 218)'
'Test dataset shape: (128726, 218)'


In [6]:
# Prepare training and test variables
X_columns = [col for col in train_df.columns if col not in ['Unnamed: 0', 'PATHOLOGY']]
X_train = train_df[X_columns]
y_train = train_df['PATHOLOGY']
X_test = test_df[X_columns]
y_test = test_df['PATHOLOGY']
pprint(f'X_train shape: {X_train.shape}')
pprint(f'y_train shape: {y_train.shape}')
pprint(f'X_test shape: {X_test.shape}')
pprint(f'y_test shape: {y_test.shape}')

'X_train shape: (982224, 216)'
'y_train shape: (982224,)'
'X_test shape: (128726, 216)'
'y_test shape: (128726,)'


In [7]:
categorical = X_train.select_dtypes("object").columns
categorical

Index(['SEX', 'Characterize your pain:', 'Do you feel pain somewhere?',
       'Does the pain radiate to another location?',
       'Have you traveled out of the country in the last 4 weeks?',
       'What color is the rash?', 'Do your lesions peel off?',
       'Where is the affected region located?',
       'Is the lesion (or are the lesions) larger than 1cm?',
       'Where is the swelling located?'],
      dtype='object')

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
label_encoder = LabelEncoder()

In [11]:
for column in categorical:
    X_train[column] = label_encoder.fit_transform(X_train[column])

In [12]:
for column in categorical:
    X_test[column] = label_encoder.fit_transform(X_test[column])

In [14]:
logistic_regression = LogisticRegression(penalty='l1', solver='liblinear')

#fit model on training data

logistic_regression.fit(X_train,y_train )

#predictions on testing data

y_pred = logistic_regression.predict(X_test)

#evaluate model

accuracy = accuracy_score(y_test, y_pred)
print(f'The accuracy score is {accuracy * 100}%')

The accuracy score is 99.3847396796296%


In [16]:
from sklearn.preprocessing import LabelBinarizer

In [17]:
lb = LabelBinarizer()

In [18]:
y_pred_encoded = lb.fit_transform(y_pred)
y_test_encoded = lb.fit_transform(y_test)

In [19]:
y_test_encoded

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
#calculate AUC of model
auc = metrics.roc_auc_score(y_test_encoded, y_pred_encoded)

#print AUC score
print(auc)

0.9959799938917276


In [21]:
print("Classification Report")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Classification Report
                                          precision    recall  f1-score   support

     Acute COPD exacerbation / infection       1.00      1.00      1.00      2153
                Acute dystonic reactions       1.00      1.00      1.00      3302
                        Acute laryngitis       0.99      0.99      0.99      3214
                      Acute otitis media       1.00      1.00      1.00      3509
                   Acute pulmonary edema       1.00      1.00      1.00      2598
                    Acute rhinosinusitis       1.00      0.67      0.80      1829
                      Allergic sinusitis       1.00      1.00      1.00      2411
                             Anaphylaxis       1.00      1.00      1.00      3798
                                  Anemia       1.00      1.00      1.00      6770
                     Atrial fibrillation       1.00      1.00      1.00      2831
                               Boerhaave       1.00      1.00      1.00    

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
X = X_train
y = y_train

In [24]:
# Took 1h6m to run
k = 5
scores = cross_val_score(logistic_regression, X, y, cv=k, scoring='accuracy')

### this took 1h6 minutes to run

In [25]:
print(f"Mean Accuracy: {np.mean(scores)}")

Mean Accuracy: 0.9938496717196672
